In [2]:
from utils import load_embedding, parse_tup, cos_sim, cos_dist
from sklearn.decomposition import PCA
from tqdm.auto import tqdm
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd

In [3]:
TIME_FRAME = "biweekly"
subreddits, vectors  = load_embedding("/h/224/cameron/Political-Subreddit-Embedding/trained_embeddings/adapted_vecs.txt",nrows=100)
embedding = pd.concat([subreddits, vectors], axis=1).set_index("subreddit")
del subreddits, vectors
embedding

,1,2,3,4,5,6,7,8,9,10,...,141,142,143,144,145,146,147,148,149,150
subreddit,,,,,,,,,,,,,,,,,,,,,
SquaredCircle,-0.006950,-0.085116,-0.018554,-0.172390,0.018786,-0.127979,-0.005660,-0.129364,0.063671,0.059154,...,0.010085,0.041204,-0.058507,-0.064563,-0.005662,0.093066,0.091820,-0.099796,0.150331,-0.085893
AskReddit,-0.012029,-0.067604,-0.003573,-0.065879,0.024166,-0.060270,-0.109945,-0.071275,0.134388,-0.074362,...,-0.007804,-0.031531,-0.198097,0.067012,-0.101151,0.092231,0.088949,-0.041761,-0.112736,-0.053113
politics,-0.133536,-0.101922,-0.034117,-0.093723,0.030623,-0.068788,0.056715,-0.093384,0.139595,-0.084614,...,-0.025986,0.084037,-0.190394,-0.032979,-0.095009,0.021711,0.044770,-0.121603,-0.009190,0.020746
memes,-0.097242,0.043635,-0.092566,0.007617,-0.065151,-0.050654,0.043962,-0.055058,-0.063219,-0.073888,...,0.043389,-0.016802,-0.127256,0.138457,-0.101161,-0.050914,0.060248,-0.158746,-0.082112,-0.052449
teenagers,-0.089968,0.044256,-0.051055,0.045911,-0.051108,-0.069145,-0.003921,-0.052291,0.030269,-0.126074,...,0.017685,-0.008737,-0.175991,0.069359,-0.126235,-0.003090,0.000458,-0.100655,-0.120615,-0.107238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pokemontrades,0.067970,-0.167568,0.123444,-0.186669,-0.005990,-0.062821,-0.035264,0.006535,0.113404,-0.077050,...,-0.048776,0.056929,-0.223332,-0.041529,-0.038856,-0.107669,0.059018,-0.061720,-0.111061,-0.111764
AnthemTheGame,-0.015490,-0.084328,-0.195951,-0.169358,0.023918,0.000886,-0.053895,-0.172507,-0.093554,-0.052629,...,0.007425,0.000070,-0.059594,-0.019920,-0.001093,0.082550,0.150585,0.005368,-0.108998,0.003355
LivestreamFail,-0.050140,0.036430,-0.079762,-0.130651,0.116921,-0.008116,0.027807,0.093914,0.117386,0.008915,...,-0.095172,0.202737,-0.084025,0.067541,0.002115,0.142261,-0.019374,-0.093588,0.017363,0.036300


In [4]:
tqdm.pandas()
def find_closest(transformation,embedding,subreddit="politics",vec=None,exclude=[]):
    # Get subreddit
    assert subreddit or vec, "Must supply a subreddit name (that's in the data frame), or the vector representation of a subreddit"
    assert subreddit in embedding.index.values, "{} is not in the dataset".format(subreddit)
    sub_vec = embedding.loc[subreddit].to_numpy().flatten() if subreddit else vec
    transformed = sub_vec + transformation
    all_other_vecs = embedding[~embedding.index.isin(exclude+[subreddit])]
    similarity = np.apply_along_axis(lambda x : cos_sim(transformed,x), 1, all_other_vecs)
    winner_ind = similarity.argmax()
    return all_other_vecs.index.values[winner_ind], similarity[winner_ind]
    
def get_transformation(sub_a,sub_ap,embedding):
    """
        sub_a is to sub_ap
        sub_ap - sub_a
        returns transformation from a to ap
    """
    assert sub_a in embedding.index.values, "{} is not in the dataset".format(sub_a)
    assert sub_ap in embedding.index.values, "{} is not in the dataset".format(sub_ap)
    vec_a = embedding.loc[sub_a].to_numpy().flatten()
    vec_ap = embedding.loc[sub_ap].to_numpy().flatten()
    return vec_ap - vec_a

def get_possible_fits(a,ap,embedding,k=10):
    trans = get_transformation(a,ap,embedding)
    possible_fits = pd.DataFrame(map(lambda x : find_closest(trans,embedding,subreddit=x,exclude=[a,ap]),tqdm(embedding.index)))
    possible_fits = possible_fits.rename({0: "subreddit_to", 1: "trans_sim"},axis=1)
    possible_fits["subreddit_from"] = embedding.index
    possible_fits = possible_fits.sort_values('trans_sim',ascending=False)
    possible_fits = possible_fits[["subreddit_from","subreddit_to","trans_sim"]].reset_index(drop=True)
    possible_fits = possible_fits[~possible_fits["subreddit_from"].isin([a,ap])]
    return possible_fits.head(k)

# a = "socialism"
# ap = "Libertarian"
# p1 = get_possible_fits(a,ap,embedding,k=20)
# print(p1)
a = "nba"
ap = "nfl"
p2 = get_possible_fits(a,ap,embedding,k=50)
print(p2)

/h/224/cameron/miniconda3/envs/pyspark_env/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel



          subreddit_from          subreddit_to  trans_sim
1               baseball                   CFB   0.742025
2               Market76                  fo76   0.733894
3         PublicFreakout  insanepeoplefacebook   0.729981
5                  memes             teenagers   0.726885
6                   news              politics   0.722581
7                   fo76              Market76   0.720759
8   PewdiepieSubmissions             teenagers   0.713107
9       unpopularopinion         AmItheAsshole   0.712579
10                  gifs                  news   0.710986
11             dankmemes                 memes   0.705230
12        wholesomememes                 memes   0.700222
13              politics                  news   0.697239
14             worldnews              politics   0.684578
15         AmItheAsshole      unpopularopinion   0.683158
16       fantasyfootball                   CFB   0.679296
17                movies                  news   0.678644
18  insanepeo

In [8]:
def analogy(sub_a,sub_ap,sub_b,embedding):
    """
        sub_a is to sub_ap, as sub_b is to ??
        sub_ap + sub_b - sub_a
        returns the subreddit that best answers the analogy
    """
    assert sub_a in embedding.index.values, "{} is not in the dataset".format(sub_a)
    assert sub_ap in embedding.index.values, "{} is not in the dataset".format(sub_ap)
    assert sub_b in embedding.index.values, "{} is not in the dataset".format(sub_b)
    vec_a = embedding.loc[sub_a].to_numpy().flatten()
    vec_ap = embedding.loc[sub_ap].to_numpy().flatten()
    vec_b = embedding.loc[sub_b].to_numpy().flatten()
    vec_bp = vec_ap + (vec_b - vec_a)
    all_other_vecs = embedding[~embedding.index.isin([sub_a,sub_ap,sub_b])]
    similarity = np.apply_along_axis(lambda x : cos_sim(x,vec_bp), 1, all_other_vecs)
    winner_ind = similarity.argmax()
    return all_other_vecs.index.values[winner_ind]

a = "socialism"
ap = "Libertarian"
b = "SandersForPresident"
print("r/{} is to r/{} as r/{} is to r/{}".format(a,ap,b,analogy(a,ap,b,embedding)))

r/socialism is to r/Libertarian as r/SandersForPresident is to r/hottiesfortrump


In [4]:
from itertools import combinations

def predict_analogies(analogies):
    tuples = [tuple(x) for x in analogies.to_numpy()]
    pairs = list(combinations(tuples,2))
    pred = []
    actual = []
    for (a,ap),(b,bp) in tqdm(pairs):
        try: 
            p = analogy(a.strip(),ap.strip(),b.strip())
            pred.append(p)
            actual.append(bp)
        except:
            pass
    return np.array(pred),np.array(actual), pairs

data_sets = ["football","basketball",""]

analogies = pd.read_csv("/h/224/cameron/Political-Subreddit-Embedding/external_datasets/validation/football.csv",nrows=50)
pred, actual, tuples = predict_analogies(analogies)
accuracy = (pred == actual).mean() * 100
num_correct = (pred == actual).sum()
print("Out of {} analogies, {} were predicted correctly ({}%)".format(len(pred),num_correct,accuracy))
for p,act,((a,ap),(b,_)) in zip(pred,actual,tuples):
    if not p==act: print("{} is to {} as {} is to {} (predicted as: {})".format(a,ap,b,act,p))



Out of 435 analogies, 245 were predicted correctly (56.32183908045977%)
Atlanta is to falcons as Dallas is to cowboys (predicted as: TexasRangers)
Atlanta is to falcons as LosAngeles is to Chargers (predicted as: Dodgers)
Atlanta is to falcons as LosAngeles is to LosAngelesRams (predicted as: Dodgers)
Atlanta is to falcons as NewOrleans is to Saints (predicted as: LSUFootball)
Atlanta is to falcons as chicago is to CHIBears (predicted as: CHICubs)
Atlanta is to falcons as houston is to Texans (predicted as: nfl)
Atlanta is to falcons as indianapolis is to Colts (predicted as: Indiana)
Atlanta is to falcons as nyc is to NYGiants (predicted as: newyorkcity)
Atlanta is to falcons as nyc is to nyjets (predicted as: newyorkcity)
Atlanta is to falcons as oakland is to oaklandraiders (predicted as: 49ers)
Atlanta is to falcons as pittsburgh is to steelers (predicted as: buccos)
Buffalo is to buffalobills as Charlotte is to panthers (predicted as: NorthCarolina)
Buffalo is to buffalobills as 